# Analysis of Fire Incidents in New York

As a part of the challenges assigned by **The Data Incubator**

## Problem Statement

The New York City Fire Department keeps a log of detailed information on incidents handled by FDNY units. In this challenge we will work with a dataset that contains a record of incidents handled by FDNY units from 2013-2017. Download the [FDNY data set](https://data.cityofnewyork.us/api/views/tm6d-hbzd/rows.csv?accessType=DOWNLOAD). Also take a look at the [dataset landing page](https://data.cityofnewyork.us/Public-Safety/Incidents-Responded-to-by-Fire-Companies/tm6d-hbzd) and find descriptions of column names [here](https://data.cityofnewyork.us/api/views/tm6d-hbzd/files/1434d09c-fbf8-4450-8b42-9fe0c3b85fb3?download=true&filename=OPEN_DATA_FIRE_INCIDENTS_FILE_DESCRIPTION.xls).

### Loading and Visualisation

First, let us load the data set. 

In [ ]:
import pandas as pd
dataset = pd.read_csv("Incidents_Responded_to_by_Fire_Companies.csv", low_memory=False)

Make a copy of the data to work on. 

In [ ]:
data = dataset.copy()

Now, let us visualize it first. 

In [ ]:
data.head()

### Cleaning up the Data

Time to clean up the data!

Looks like not all the columns are not going to be useful for the exercise. I drop them here. 

In [ ]:
data = data.drop(columns=['STANDPIPE_SYS_PRESENT_FLAG', 'AES_PRESENCE_DESC', 'DETECTOR_PRESENCE_DESC',
                          'FIRE_SPREAD_DESC', 'STORY_FIRE_ORIGIN_COUNT', 'FIRE_ORIGIN_BELOW_GRADE_FLAG', 'FLOOR',
                          'STREET_HIGHWAY', 'ACTION_TAKEN2_DESC', 'ACTION_TAKEN3_DESC', 'PROPERTY_USE_DESC'])

Few columns have alphanumneric data. I'm only interested in the numeric data here; I can always look up what they mean later. 

In [ ]:
data['INCIDENT_TYPE_DESC']= data['INCIDENT_TYPE_DESC'].str.split(pat = "-").str[0] 
data['HIGHEST_LEVEL_DESC'] = data['HIGHEST_LEVEL_DESC'].str.split(pat = "-").str[0]
data['ACTION_TAKEN1_DESC'] = data['ACTION_TAKEN1_DESC'].str.split(pat = "-").str[0]
data['BOROUGH_DESC'] = data['BOROUGH_DESC'].str.split(pat = "-").str[0]
data['ZIP_CODE'] = data['ZIP_CODE'].str.split(pat = "-").str[0]

In [ ]:
#This probably is useful as numeric data. 

data['INCIDENT_TYPE_DESC'] = data['INCIDENT_TYPE_DESC'].str.split(pat = "B").str[0]
data['INCIDENT_TYPE_DESC'] = data['INCIDENT_TYPE_DESC'].str.split(pat = "A").str[0]

We have three columns that have `datetime` data. I use the `infer_datetime_format=True` option to speed up the function. 

In [ ]:
# Convert Datetime data

%timeit data['INCIDENT_DATE_TIME'] = pd.to_datetime(data.INCIDENT_DATE_TIME, infer_datetime_format=True)
%timeit data['ARRIVAL_DATE_TIME'] = pd.to_datetime(data.ARRIVAL_DATE_TIME, infer_datetime_format=True)
%timeit data['LAST_UNIT_CLEARED_DATE_TIME'] = pd.to_datetime(data.LAST_UNIT_CLEARED_DATE_TIME, 
                                                             infer_datetime_format=True)

### Exporting Clean Data

Good! Looks like we have cleaned up our data now. Let's visualize it first and then save it into a .csv that I can import later and start working from there. 

In [ ]:
data.head()

In [ ]:
data.to_csv("clean_data.csv", index=False)

In [ ]:
# Can start from here later. 

import pandas as pd

data_mod = pd.read_csv("clean_data.csv", low_memory=False)
#                        , parse_dates=['INCIDENT_DATE_TIME ', 'ARRIVAL_DATE_TIME', 'LAST_UNIT_CLEARED_DATE_TIME']
#                        , infer_datetime_format=True)

In [ ]:
data = data_mod.copy()

In [ ]:
data.head()

In [ ]:
# Convert Datetime data

%timeit data['INCIDENT_DATE_TIME'] = pd.to_datetime(data.INCIDENT_DATE_TIME, infer_datetime_format=True)
%timeit data['ARRIVAL_DATE_TIME'] = pd.to_datetime(data.ARRIVAL_DATE_TIME, infer_datetime_format=True)
%timeit data['LAST_UNIT_CLEARED_DATE_TIME'] = pd.to_datetime(data.LAST_UNIT_CLEARED_DATE_TIME, infer_datetime_format=True)


In [ ]:
data.dtypes

## Problems

### 1. What proportion of FDNY responses in this dataset correspond to the most common type of incident?

In [ ]:
data['INCIDENT_TYPE_DESC'].value_counts()[:5]

In [ ]:
most_freq_incident = (data['INCIDENT_TYPE_DESC'] == 412).sum()
most_freq_incident

Look like the most common incident is No. 412, happening 102410 times

In [ ]:
count_row, count_column = data.shape

In [ ]:
#Total number of data points. 

count_row

In [ ]:
#Proportion of most common incident

prop_1 = most_freq_incident/count_row
prop_1

### 2. What is the ratio of the average number of units that arrive to a scene of an incident classified as '111 - Building fire' to the number that arrive for '651 - Smoke scare, odor of smoke'?

In [ ]:
sel1 = data.loc[(data['INCIDENT_TYPE_DESC'] == 111)]
sel1.head()

In [ ]:
sel2 = data.loc[(data['INCIDENT_TYPE_DESC'] == 651)]
sel2.head()

In [ ]:
mean_111 = sel1['UNITS_ONSCENE'].mean()
mean_651 = sel2['UNITS_ONSCENE'].mean()

prop_2 = mean_111/mean_651
prop_2

### 3. How many times more likely is an incident in Staten Island a false call compared to in Manhattan? The answer should be the ratio of Staten Island false call rate to Manhattan false call rate. A false call is an incident for which 'INCIDENT_TYPE_DESC' is '710 - Malicious, mischievous false call, other'.

In [ ]:
sel3 = data.loc[(data['INCIDENT_TYPE_DESC'] == 710)]
sel3.head()
sel3.shape 

So, we don't have many false calls anyway. 

There was some information lost in converting the last row (BOROUGH_DESC) to just numbers. I add context here: 

1. Manhattan
2. Bronx
3. Staten Island
4. Brooklyn
5. Queens

In [ ]:
manhattan_false_calls = (sel3['BOROUGH_DESC'] == 1).sum()

In [ ]:
manhattan_false_calls

In [ ]:
staten_island_false_calls = (sel3['BOROUGH_DESC'] == 3).sum()

In [ ]:
staten_island_false_calls

In [ ]:
prop_3= staten_island_false_calls/manhattan_false_calls
prop_3

### 4. Check the distribution of the number of minutes it takes between the time a '111 - Building fire' incident has been logged into the Computer Aided Dispatch system and the time at which the first unit arrives on scene. What is the third quartile of that distribution. Note: the number of minutes can be fractional (ie, do not round).

In [ ]:
sel1.head()
pd.options.mode.chained_assignment = None #Reomve the flag for appending a new column. 

In [ ]:
sel1['ARRIVAL_TIME'] = sel1['ARRIVAL_DATE_TIME'] - sel1['INCIDENT_DATE_TIME']

In [ ]:
#Converting this to numerical minutes. 

sel1['ARRIVAL_TIME_IN_MINS'] = sel1['ARRIVAL_TIME'].dt.total_seconds().div(60).astype(float)

In [ ]:
sel1.head()

In [ ]:
#Plotting the distribution

%matplotlib inline
import matplotlib.pyplot as plt

sel1['ARRIVAL_TIME_IN_MINS'].hist()

In [ ]:
# The third quartile can be found using the "decribe" function, 

sel1['ARRIVAL_TIME_IN_MINS'].describe()

In [ ]:
#Or, using, 

prop_4 = sel1['ARRIVAL_TIME_IN_MINS'].quantile(0.75)
prop_4

Well, the answers match anyway. 

### 5. We can use the FDNY dataset to investigate at what time of the day people cook most. Compute what proportion of all incidents are cooking fires for every hour of the day by normalizing the number of cooking fires in a given hour by the total number of incidents that occured in that hour. Find the hour of the day that has the highest proportion of cooking fires and submit that proportion of cooking fires. A cooking fire is an incident for which 'INCIDENT_TYPE_DESC' is '113 - Cooking fire, confined to container'. Note: round incident times down. For example, if an incident occured at 22:55 it occured in hour 22.

In [ ]:
data['INCIDENT_DATE_TIME'].head()

In [ ]:
# Selecting the rounded off hour

data['INCIDENT_HOUR'] = data['INCIDENT_DATE_TIME'].dt.hour

In [ ]:
#Count incidents per hour

incidents_per_hour = data['INCIDENT_HOUR'].value_counts()

In [ ]:
incidents_per_hour_sorted = incidents_per_hour.sort_index()
incidents_per_hour_sorted.head()

`incidents_per_hour_sorted` gives the number of incidents every hour. Let's narrow down our data to Cooking incidents. 

In [ ]:
sel4 = data.loc[(data['INCIDENT_TYPE_DESC'] == 113)]
sel4.head()

In [ ]:
#Doing the same analysis as above for cooking incidents, 

cooking_incidents_per_hour = sel4['INCIDENT_HOUR'].value_counts()

In [ ]:
cooking_incidents_per_hour_sorted = cooking_incidents_per_hour.sort_index()
cooking_incidents_per_hour_sorted.head()

In [ ]:
ratio_of_cooking_incidents = cooking_incidents_per_hour_sorted/incidents_per_hour_sorted
prop_5 = max(ratio_of_cooking_incidents)
prop_idx = ratio_of_cooking_incidents.idxmax()
print(prop_5,prop_idx)

### 6. What is the coefficient of determination (R squared) between the number of residents at each zip code and the number of inicidents whose type is classified as '111 - Building fire' at each of those zip codes. Note: The 2010 US Census population by zip code dataset should be downloaded from [here](https://s3.amazonaws.com/SplitwiseBlogJB/2010+Census+Population+By+Zipcode+(ZCTA).csv). You will need to use both the FDNY responses and the US Census dataset. Ignore zip codes that do not appear in the census table.

In [ ]:
census_data = pd.read_csv("2010+Census+Population+By+Zipcode+(ZCTA).csv")

In [ ]:
census_data = census_data.rename(index=str, columns={"Zip Code ZCTA": "ZIP_CODE", "2010 Census Population": "POPULATION"})

In [ ]:
census_data.head()

In [ ]:
#Reload fire incidents

sel1 = data.loc[(data['INCIDENT_TYPE_DESC'] == 111)]
sel1.head()

In [ ]:
#Grouping data by zip code

zip_codes = sel1['ZIP_CODE'].value_counts()

In [ ]:
zip_codes.head()

In [ ]:
zip_codes = zip_codes.rename_axis('ZIP_CODE').reset_index(name='NUMBER_OF_FIRE_INCIDENTS')

Dropping indices where Zip Code is 9999: 

In [ ]:
index_zip_99999 = zip_codes[zip_codes.ZIP_CODE == 99999].index
zip_codes = zip_codes.drop(index_zip_99999)

In [ ]:
zip_codes.head()

In [ ]:
joined_df = zip_codes.merge(census_data)

In [ ]:
joined_df.head()

In [ ]:
#Phew, now let's find out the Co-efficient of Determination (R^2)

from sklearn.metrics import r2_score

prop_6 = r2_score(joined_df['POPULATION'], joined_df['NUMBER_OF_FIRE_INCIDENTS'])

In [ ]:
prop_6

### 7. For this question, only consider incidents that have information about whether a CO detector was present or not. We are interested in how many times more likely it is that an incident is long when no CO detector is present compared to when a CO detector is present. For events with CO detector and for those without one, compute the proportion of incidents that lasted 20-30, 30-40, 40-50, 50-60, and 60-70 minutes (both interval boundary values included) by dividing the number of incidents in each time interval with the total number of incidents. For each bin, compute the ratio of the 'CO detector absent' frequency to the 'CO detector present' frequency. Perform a linear regression of this ratio to the mid-point of the bins. From this, what is the predicted ratio for events lasting 39 minutes?

In [ ]:
sel5 = data.copy()

In [ ]:
# Remove all rows where we don't have information whether a CO detector was present or not. 

sel5 = sel5.dropna()

`TOTAL_INCIDENT_DURATION` is in seconds. Converting to minutes first. 

In [ ]:
sel5['TOTAL_INCIDENT_DURATION'] = sel5['TOTAL_INCIDENT_DURATION']/60.0

In [ ]:
sel5.head()

In [ ]:
bin_total_incidents = pd.cut(sel5['TOTAL_INCIDENT_DURATION'], [20.0, 30.0, 40.0, 50.0, 60.0, 70.0])
total_incidents = bin_total_incidents.value_counts()
total_incidents

In [ ]:
sel6 = sel5.loc[(data['CO_DETECTOR_PRESENT_DESC'] == 'Yes')]
sel6.head()

In [ ]:
sel7 = sel5.loc[(data['CO_DETECTOR_PRESENT_DESC'] == 'No')]
sel7.head()

In [ ]:
bins_yes = pd.cut(sel6['TOTAL_INCIDENT_DURATION'], [20.0, 30.0, 40.0, 50.0, 60.0, 70.0])
co_detector_present = bins_yes.value_counts()
co_detector_present

In [ ]:
bins_no = pd.cut(sel7['TOTAL_INCIDENT_DURATION'], [20.0, 30.0, 40.0, 50.0, 60.0, 70.0])
co_detector_absent = bins_no.value_counts()
co_detector_absent

First, need to normalise and then compute proprtion of events in the Yes and No bin out of the total incidents. 

In [ ]:
normalised_yes = co_detector_present/total_incidents
normalised_yes

In [ ]:
normalised_no = co_detector_absent/total_incidents
normalised_no

In [ ]:
normalised_ratio_of_co = normalised_yes/normalised_no
normalised_ratio_of_co

In [ ]:
normalised_ratio_of_co = normalised_ratio_of_co.rename_axis('BINS').reset_index(name='RATIO_OF_INCIDENTS')

In [ ]:
normalised_ratio_of_co.head()

In [ ]:
normalised_ratio_of_co.dtypes

Now, let us perform the Linear Regression. 

In [ ]:
bins_yes.plot.scatter(x = "" ,y = "TOTAL_INCIDENT_DURATION")


In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(normalised_ratio_of_co)